In [2]:
import spacy    
from spacy.lang.en.examples import sentences      
import en_core_web_md                

In [3]:
spacy_model = en_core_web_md.load()

In [4]:
doc = spacy_model(sentences[1])
doc

Autonomous cars shift insurance liability toward manufacturers

In [5]:
doc.text

'Autonomous cars shift insurance liability toward manufacturers'

In [6]:
for token in doc:
    print(token.text, token.pos_, token.dep_)

Autonomous ADJ amod
cars NOUN nsubj
shift VERB ROOT
insurance NOUN compound
liability NOUN dobj
toward ADP prep
manufacturers NOUN pobj


In [7]:
for token1 in doc:
    for token2 in doc:
        print(token1.text, token2.text, token1.similarity(token2))

Autonomous Autonomous 1.0
Autonomous cars 0.17233114
Autonomous shift 0.17754015
Autonomous insurance 0.105331734
Autonomous liability 0.15329868
Autonomous toward 0.21580476
Autonomous manufacturers 0.10932432
cars Autonomous 0.17233114
cars cars 1.0
cars shift 0.26758885
cars insurance 0.41793215
cars liability 0.21896149
cars toward 0.22643843
cars manufacturers 0.35537532
shift Autonomous 0.17754015
shift cars 0.26758885
shift shift 1.0
shift insurance 0.19694921
shift liability 0.14714341
shift toward 0.40878925
shift manufacturers 0.1890121
insurance Autonomous 0.105331734
insurance cars 0.41793215
insurance shift 0.19694921
insurance insurance 1.0
insurance liability 0.6519564
insurance toward 0.21364474
insurance manufacturers 0.31751585
liability Autonomous 0.15329868
liability cars 0.21896149
liability shift 0.14714341
liability insurance 0.6519564
liability liability 1.0
liability toward 0.18401146
liability manufacturers 0.30955592
toward Autonomous 0.21580476
toward cars 0

In [8]:
#Out of vocabulary
# oov => out of vocabulary

check = spacy_model('jsbh')
for c in check:
    print(c.is_oov)

True


In [9]:
check = spacy_model('crackers')
for c in check:
    print(c.is_oov)

False


In [10]:
import pandas as pd
import numpy as np
from keras import regularizers, optimizers
from keras.layers.experimental.preprocessing import TextVectorization
from keras.layers import Embedding, Dense, Dropout, Input, LSTM, GlobalMaxPool1D
from keras.models import Sequential
from keras.initializers import Constant
import tensorflow as tf

In [11]:
Vectorizer = TextVectorization()

In [12]:
dataset = pd.read_csv('valid.csv')
dataset.head()

,Id,Title,Body,Tags,CreationDate,Y
0,34552974,How to get all the child records from differen...,I am having 4 different tables like \r\nselect...,<sql><sql-server>,2016-01-01 01:44:52,LQ_EDIT
1,34554721,Retrieve all except some data of the another t...,I have two table m_master and tbl_appointment\...,<php><mysql><sql><codeigniter><mysqli>,2016-01-01 08:43:50,LQ_EDIT
2,34555135,Pandas: read_html,<p>I'm trying to extract US states from wiki U...,<python><pandas>,2016-01-01 09:55:22,HQ
3,34555448,Reader Always gimme NULL,"I'm so new to C#, I wanna make an application ...",<sql-server><c#-4.0>,2016-01-01 10:43:45,LQ_EDIT
4,34555752,php rearrange array elements based on condition,basically i have this array:\r\n\r\n array(...,<php>,2016-01-01 11:34:09,LQ_EDIT


In [30]:
X_train = dataset.loc[dataset["Tags"]=="<python>", "Title"]
X_train.head()

19     i am new to pythn and was trying to fix indent...
375                   Python syntax error in identation?
481                                  Program doesn't run
677    list index is out of range - cannot find why -...
691                            Opening a .ipynb.txt File
Name: Title, dtype: object

In [31]:
Y_train = dataset['Id']
Y_train.head()

0    34552974
1    34554721
2    34555135
3    34555448
4    34555752
Name: Id, dtype: int64

In [32]:
Vectorizer.adapt(X_train.to_numpy())

In [33]:
Vectorizer

In [34]:
vocab = Vectorizer.get_vocabulary()
vocab

['',
 '[UNK]',
 'python',
 'in',
 'to',
 'a',
 'the',
 'how',
 'of',
 'list',
 'and',
 'i',
 'is',
 'error',
 'this',
 'with',
 'for',
 'number',
 'code',
 'from',
 'function',
 'why',
 'my',
 'loop',
 'string',
 'what',
 'using',
 'on',
 'get',
 'do',
 'not',
 'file',
 'does',
 'create',
 'that',
 'if',
 'am',
 'values',
 'syntax',
 'make',
 'it',
 'data',
 'an',
 'text',
 'or',
 'input',
 'work',
 'while',
 'numbers',
 'multiple',
 'help',
 'find',
 'can',
 'are',
 'add',
 'wrong',
 'typeerror',
 'object',
 'new',
 'line',
 'have',
 'be',
 '2',
 'variable',
 'time',
 'statement',
 'return',
 'program',
 'print',
 'please',
 'output',
 'lists',
 'into',
 'getting',
 'files',
 'by',
 'but',
 'at',
 '0',
 'you',
 'want',
 'variables',
 'value',
 'type',
 'trying',
 'script',
 'range',
 'random',
 'no',
 'n',
 'me',
 'like',
 'invalid',
 'game',
 'convert',
 'class',
 'array',
 'write',
 'when',
 'user',
 'use',
 'two',
 'tuple',
 'there',
 'system',
 'someone',
 'some',
 'simple',
 'rea

# Embedded Matrix

In [35]:
total_token = len(vocab)
total_token

744

In [36]:
from nltk.tokenize import word_tokenize

In [37]:
word_count = lambda X_train: len(vocab)
embedding_dim = max(X_train, key = word_count)
len(word_tokenize(embedding_dim))

23

In [38]:
embedding_matrix = np.zeros((total_token, 300))

In [39]:
for i, word in enumerate(vocab):
    embedding_matrix[i] = spacy_model(word).vector

In [40]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.203725  , -0.083137  ,  0.20306967, ...,  0.22994667,
        -0.19698666, -0.08194   ],
       [ 0.035414  , -0.45730001,  0.42616999, ...,  0.06664   ,
        -0.43283999,  0.14802   ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.17911001, -0.15216   ,  0.053769  , ...,  0.062401  ,
        -0.36908001,  0.71759999],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [41]:
len(embedding_matrix)

744

In [42]:
Embedding_layer=Embedding(
    total_token,
    300,
    embeddings_initializer=Constant(embedding_matrix),
    trainable=False)

In [43]:
Embedding_layer
